In [148]:
import json
import requests

import pandas as pd
from pandas.io.json import json_normalize
from sodapy import Socrata


In [149]:
pd.options.display.max_columns=400

In [138]:
socrata_pswd = 
socrata_key = 
socrata_username = 

In [150]:
# Read in the MV104 - Vehicle Table from the Open Data Portal
request = Socrata('nycopendata.socrata.com', socrata_key,
               username = socrata_username,
               password = socrata_pswd)

ai_uid = 'djiz-puhd'
crashes_api = request.get(ai_uid, limit=2000000)
crashes_df = pd.DataFrame(crashes_api)
crashes_df.head()

,cft_cde1,cft_cde2,cs_id,cv_cylndr,cv_id,cv_psgr_num,cv_reg_state_cde,cv_veh_year,cv_vehmake_descr,cv_weight_lbs,dirctt_cde,dmv_vin_num,ft_id,paccactt_id,regt_cde,shzmtt_id,tbct_dmv_cde,veh_evntt_id,vehbdyt_id,vin
0,NaN,NaN,32045885,4,5979599,-1,NY,1998,TOYOT,NaN,-1,NaN,1,-1,16,NaN,-3,NaN,6,2T1BR12E2WC
1,NaN,NaN,32045886,NaN,5979600,3,SC,NaN,NaN,NaN,-1,NaN,-3,3,-3,NaN,-3,NaN,6,NaN
2,NaN,NaN,32045886,6,5979601,2,NY,1999,DODGE,NaN,-1,NaN,1,1,16,NaN,-3,NaN,5,2B4FP2532XR
3,NaN,NaN,32045887,8,5979602,-1,NY,2002,GMC,NaN,-1,NaN,2,1,56,NaN,-3,NaN,60,1GDJG31F821
4,NaN,NaN,32045887,NaN,5979603,1,OK,NaN,NaN,NaN,-1,NaN,-3,1,-3,NaN,-3,NaN,15,NaN


In [140]:
# Remove null vins
crashes_df = crashes_df[pd.notnull(crashes_df['vin'])]
crashes_df.head()

,cft_cde1,cft_cde2,cs_id,cv_cylndr,cv_id,cv_psgr_num,cv_reg_state_cde,cv_veh_year,cv_vehmake_descr,cv_weight_lbs,dirctt_cde,dmv_vin_num,ft_id,paccactt_id,regt_cde,shzmtt_id,tbct_dmv_cde,veh_evntt_id,vehbdyt_id,vin
0,NaN,NaN,32045885,4,5979599,-1,NY,1998,TOYOT,NaN,-1,NaN,1,-1,16,NaN,-3,NaN,6,2T1BR12E2WC
2,NaN,NaN,32045886,6,5979601,2,NY,1999,DODGE,NaN,-1,NaN,1,1,16,NaN,-3,NaN,5,2B4FP2532XR
3,NaN,NaN,32045887,8,5979602,-1,NY,2002,GMC,NaN,-1,NaN,2,1,56,NaN,-3,NaN,60,1GDJG31F821
6,NaN,NaN,32046208,4,5980144,-1,NY,1996,SUBAR,NaN,-1,NaN,1,10,29,NaN,-3,NaN,5,4S3BK6657T7
7,-1,NaN,32046209,8,5980145,1,NY,1997,MERCU,NaN,W,NaN,1,1,16,NaN,-3,NaN,5,4M2DU55P6VU


In [141]:
crashes_df['cv_veh_year'] = crashes_df['cv_veh_year'].astype(str)
crashes_df['vin'] = crashes_df['vin'].astype(str)

In [142]:
# Take a random sample of crashes_df
crashes_sample = crashes_df.sample(n=10000)
crashes_sample.head()

,cft_cde1,cft_cde2,cs_id,cv_cylndr,cv_id,cv_psgr_num,cv_reg_state_cde,cv_veh_year,cv_vehmake_descr,cv_weight_lbs,dirctt_cde,dmv_vin_num,ft_id,paccactt_id,regt_cde,shzmtt_id,tbct_dmv_cde,veh_evntt_id,vehbdyt_id,vin
161309,NaN,NaN,32529625,8,6897949,1,NY,2004,FORD,NaN,N,NaN,1,1,54,NaN,-3,NaN,63,2FAFP70W54X
502406,NaN,NaN,33418720,6,8571462,2,NY,2006,CHEVR,NaN,-1,NaN,1,8,16,NaN,-3,NaN,6,2G1WC581469
822416,-2,NaN,34360573,6,10362973,6,NY,2006,HONDA,NaN,S,NaN,1,7,16,NaN,-3,-2,5,2HKYF18416H
795881,-1,NaN,34266777,6,10180092,-1,NY,2005,HONDA,NaN,SW,NaN,1,1,16,NaN,-3,NaN,7,1HGCM82665A
97167,-2,NaN,32340532,2,6555274,1,NY,1995,HARLE,NaN,E,NaN,1,1,36,NaN,-3,NaN,10,1HD1BNL1XSY


In [143]:
# Read in the vehicle information using the NHTSA vehicle api 
vin_api_data = pd.DataFrame()

for index, row in crashes_sample.iterrows():
    vin = row['vin']
    year = row['cv_veh_year']
    vin_api = requests.get('https://vpic.nhtsa.dot.gov/api/vehicles/decodevinvalues/'+vin+'*BA?format=json&modelyear='+year)
    vin_data = vin_api.json()
    vin_df = pd.DataFrame(vin_data)
    # Untangle dictionary within dataframr to get results
    vin_normalize = json_normalize(vin_df['Results'])
    # Add each line of results (individual API call) to vin_api_data 
    vin_api_data = pd.concat([vin_api_data, vin_normalize], axis=0)

vin_api_data.head()



,ABS,AEB,ActiveSafetySysNote,AdaptiveCruiseControl,AdaptiveDrivingBeam,AdditionalErrorText,AirBagLocCurtain,AirBagLocFront,AirBagLocKnee,AirBagLocSeatCushion,AirBagLocSide,Artemis,AutoReverseSystem,AutomaticPedestrainAlertingSound,AxleConfiguration,Axles,BasePrice,BatteryA,BatteryA_to,BatteryCells,BatteryInfo,BatteryKWh,BatteryKWh_to,BatteryModules,BatteryPacks,BatteryType,BatteryV,BatteryV_to,BedLengthIN,BedType,BlindSpotMon,BodyCabType,BodyClass,BrakeSystemDesc,BrakeSystemType,BusFloorConfigType,BusLength,BusType,CAFEBodyType,CAFEMake,CAFEModel,CAN_AACN,CashForClunkers,ChargerLevel,ChargerPowerKW,CoolingType,Country,CurbWeightLB,CustomMotorcycleType,DaytimeRunningLight,DestinationMarket,DisplacementCC,DisplacementCI,DisplacementL,Doors,DriveType,DynamicBrakeSupport,EDR,ESC,EVDriveUnit,ElectrificationLevel,EngineConfiguration,EngineCycles,EngineCylinders,EngineHP,EngineHP_to,EngineKW,EngineManufacturer,EngineModel,EntertainmentSystem,EquipmentType,ErrorCode,ForwardCollisionWarning,FuelInjectionType,FuelTypePrimary,FuelTypeSecondary,GVWR,KeylessIgnition,LaneDepartureWarning,LaneKeepSystem,LowerBeamHeadlampLightSource,Make,Manufacturer,ManufacturerId,ManufacturerType,Model,ModelYear,MotorcycleChassisType,MotorcycleSuspensionType,NCAPBodyType,NCAPMake,NCAPModel,NCICCode,NCSABodyType,NCSAMake,NCSAModel,Note,OtherBusInfo,OtherEngineInfo,OtherMotorcycleInfo,OtherRestraintSystemInfo,OtherTrailerInfo,ParkAssist,PedestrianAutomaticEmergencyBraking,PlantCity,PlantCompanyName,PlantCountry,PlantState,PossibleValues,Pretensioner,RearVisibilityCamera,SAEAutomationLevel,SAEAutomationLevel_to,SeatBeltsAll,SeatRows,Seats,SemiautomaticHeadlampBeamSwitching,Series,Series2,SteeringLocation,SuggestedVIN,TPMS,TopSpeedMPH,TrackWidth,TractionControl,TrailerBodyType,TrailerLength,TrailerType,TransmissionSpeeds,TransmissionStyle,Trim,Trim2,Turbo,VIN,ValveTrainDesign,VehicleType,WheelBaseLong,WheelBaseShort,WheelBaseType,WheelSizeFront,WheelSizeRear,Wheels,Windows
0,,,,,,,,1st Row (Driver & Passenger),,,,,,,,,,,,,,,,,,,,,,,,,Sedan/Saloon,,,,,,,,,,,,,,,,,,,4600.0,280.70922283576,4.6,4,,,,,,,V-Shaped,,8,224,239,167.0368,Ford,,,,6 - Incomplete VIN.,,,Gasoline,,,,,,,FORD,"FORD MOTOR COMPANY OF CANADA, LTD.",977,,Crown Victoria,2004,,,,,,,,,,,,,,,,,,Talbotville,St. Thomas,Canada,Ontario,,,,,,Manual,,,,,,,,,,,,,,,,,Fleet,,,2FAFP70W54X*BA,Single Overhead Cam (SOHC),PASSENGER CAR,,,Long,,,,
0,,,,,,,1st Row (Driver & Passenger),1st Row (Driver & Passenger),,,,,,,,,,,,,,,,,,,,,,,,,Sedan/Saloon,,,,,,,,,,,,,,,,,,,3900.0,237.99260196945,3.9,4,,,,,,,V-Shaped,,6,,,,,LZ9,,,6 - Incomplete VIN.,,Sequential Fuel Injection (SFI),Gasoline,,,,,,,CHEVROLET,GENERAL MOTORS LLC,984,,Impala,2006,,,,,,,,,,,,"LZ9: 3.9L, V6, SFI",,Has Occupant Sensor,,,,Oshawa #1 (OSH),GMNA,Canada,ON,,,,,,Manual,,,,Lt Sedan,,,,,,,,,,,,,,,,2G1WC581469*BA,,PASSENGER CAR,,,,,,,4
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Sport Utility Vehicle (SUV)/Multi Purpose Vehi...,,Hydraulic,,,,,,,,,,,,,,,,,3474.057568,212,3.474057568,5,4WD/4-Wheel Drive/4x4,,,,,,V-Shaped,,6,244,,181.9508,,J35A9,,,6 - Incomplete VIN.,,,Gasoline,,"Class 1D: 5,001 - 6,000 lb (2,268 - 2,722 kg)",,,,,HONDA,"HONDA OF CANADA MFG., INC.",990,,Pilot,2006,,,,,,,,,,,,,,,,,,Alliston,,Canada,Ontario,,,,,,,,,,,,,,,,,,,,,5,Automatic,EX,,,2HKYF18416H*BA,Single Overhead Cam (SOHC),MULTIPURPOSE PASSENGER VEHICLE (MPV),,,,,,,
0,,,,,,,1st & 2nd Rows,1st Row (Driver & Passenger),,,1st Row (Driver & Passenger),,,,,,,,,,,,,,,,,,,,,,Coupe,,,,,,,,,,,,,,,,,,,2998.832712,183,2.998832712,2,,,,,,,V-Shaped,,6,240,,178.9680,,J30A4,,,6 - Incomplete VIN.,,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",,,,,HONDA,"HONDA OF AMERICA MFG., INC.",988,,Accord,2005,,,,,,,,,,,,,,Seat Belt (Rr center position),,,,Marysville,,United States (USA),Ohio,,,,,,Manual,,,,,,,,,,,,,,,5,Automatic,EX-V6,,,1HGCM82665A*BA,Single Overhead Cam (SOHC),PASSENGER CAR,,,,,,,
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Motorcycle - Cruiser,,,,,,,,,,,,,,,,,,,1340,81.771817086,1.340000,,,,,,,,V-Sha

In [144]:
# Take first 11 digits of the API result's vin
vin_api_data['partial_vin'] = vin_api_data['VIN'].str[:11]

In [145]:
# Use that 11 digit vin to join with the original vehicle table
complete_car_info = vin_api_data.set_index('partial_vin').join(crashes_sample.set_index('vin'))
complete_car_info.head()

,ABS,AEB,ActiveSafetySysNote,AdaptiveCruiseControl,AdaptiveDrivingBeam,AdditionalErrorText,AirBagLocCurtain,AirBagLocFront,AirBagLocKnee,AirBagLocSeatCushion,AirBagLocSide,Artemis,AutoReverseSystem,AutomaticPedestrainAlertingSound,AxleConfiguration,Axles,BasePrice,BatteryA,BatteryA_to,BatteryCells,BatteryInfo,BatteryKWh,BatteryKWh_to,BatteryModules,BatteryPacks,BatteryType,BatteryV,BatteryV_to,BedLengthIN,BedType,BlindSpotMon,BodyCabType,BodyClass,BrakeSystemDesc,BrakeSystemType,BusFloorConfigType,BusLength,BusType,CAFEBodyType,CAFEMake,CAFEModel,CAN_AACN,CashForClunkers,ChargerLevel,ChargerPowerKW,CoolingType,Country,CurbWeightLB,CustomMotorcycleType,DaytimeRunningLight,DestinationMarket,DisplacementCC,DisplacementCI,DisplacementL,Doors,DriveType,DynamicBrakeSupport,EDR,ESC,EVDriveUnit,ElectrificationLevel,EngineConfiguration,EngineCycles,EngineCylinders,EngineHP,EngineHP_to,EngineKW,EngineManufacturer,EngineModel,EntertainmentSystem,EquipmentType,ErrorCode,ForwardCollisionWarning,FuelInjectionType,FuelTypePrimary,FuelTypeSecondary,GVWR,KeylessIgnition,LaneDepartureWarning,LaneKeepSystem,LowerBeamHeadlampLightSource,Make,Manufacturer,ManufacturerId,ManufacturerType,Model,ModelYear,MotorcycleChassisType,MotorcycleSuspensionType,NCAPBodyType,NCAPMake,NCAPModel,NCICCode,NCSABodyType,NCSAMake,NCSAModel,Note,OtherBusInfo,OtherEngineInfo,OtherMotorcycleInfo,OtherRestraintSystemInfo,OtherTrailerInfo,ParkAssist,PedestrianAutomaticEmergencyBraking,PlantCity,PlantCompanyName,PlantCountry,PlantState,PossibleValues,Pretensioner,RearVisibilityCamera,SAEAutomationLevel,SAEAutomationLevel_to,SeatBeltsAll,SeatRows,Seats,SemiautomaticHeadlampBeamSwitching,Series,Series2,SteeringLocation,SuggestedVIN,TPMS,TopSpeedMPH,TrackWidth,TractionControl,TrailerBodyType,TrailerLength,TrailerType,TransmissionSpeeds,TransmissionStyle,Trim,Trim2,Turbo,VIN,ValveTrainDesign,VehicleType,WheelBaseLong,WheelBaseShort,WheelBaseType,WheelSizeFront,WheelSizeRear,Wheels,Windows,cft_cde1,cft_cde2,cs_id,cv_cylndr,cv_id,cv_psgr_num,cv_reg_state_cde,cv_veh_year,cv_vehmake_descr,cv_weight_lbs,dirctt_cde,dmv_vin_num,ft_id,paccactt_id,regt_cde,shzmtt_id,tbct_dmv_cde,veh_evntt_id,vehbdyt_id
1102870009*,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,7 - Manufacturer is not registered with NHTSA ...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1102870009*BA,,,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11612012015,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,7 - Manufacturer is not registered with NHTSA ...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,11612012015*BA,,,,,,,,,,7,18,33607468,6,8939949,1,NY,1980,ME/BE,NaN,S,NaN,1,3,16,NaN,-3,NaN,6
19UUA5643XA,,,,,,,,1st Row (Driver & Passenger),,,,,,,,,,,,,,,,,,,,,,,,,Sedan/Saloon,,,,,,,,,,,,,,,,,,,3211.864544,196,3.2,4,,,,,,,V-Shaped,,6,225,,167.7825,,J32A1,,,6 - Incomplete VIN.,,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",,,,,ACURA,"HONDA OF AMERICA MFG., INC.",988,,TL,1999,,,,,,,,,,,,,,,,,,Marysville,,United States (USA),Ohio,,,,,,Manual,,,,,,,,,,,,,,,4,Automatic,3.2,BASE,,19UUA5643XA*BA,Single Overhead Cam (SOHC),PASSENGER CAR,,,,,,,,NaN,NaN,32573643,6,6979908,-1,NY,1999,ACURA,NaN,W,NaN,1,1,16,NaN,-3,NaN,6
19UUA564XXA,,,,,,,,1st Row (Driver & Passenger),,,,,,,,,,,,,,,,,,,,,,,,,Sedan/Saloon,,,,,,,,,,,,,,,,,,,3211.864544,196,3.2,4,,,,,,,V-Shaped,,6,225,,167.7825,,J32A1,,,6 - Incomplete VIN.,,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",,,,,ACURA,"HONDA OF AMERICA MFG., INC.",988,,TL,1999,,,,,,,,,,,,,,,,,,Marysville,,United States (USA),Ohio,,,,,,Manual,,,,,,,,,,,,,,,4,Automatic,3.2,BASE,,19UUA564XXA*BA,Single Overhead Cam (SOHC),PASSENGER CAR,,,,,,,,26,NaN,34208688,6,10069964,1,NY,1999,ACURA,NaN,W,NaN,1,1,16,NaN,-3,NaN,6
19UUA5653XA,,,,,,,,1st Row (Driver & Passenger),,,,,,,,,,,,,,,,,,,,,,,,,Sedan/Saloon,,,,,,,,,,,,,,,,,,,3211.864544,196,3.211864544,4,,,,,,,V-Shaped,,6,225,,167.78

In [146]:
# Compare body ID with API vehicle type 
complete_car_info.groupby(['vehbdyt_id','VehicleType'])[['VIN']].count()

VIN
vehbdyt_id VehicleType                               
1          PASSENGER CAR                            1
10                                                  1
           MOTORCYCLE                             117
13                                                  2
15         INCOMPLETE VEHICLE                       1
22         INCOMPLETE VEHICLE                      13
23                                                  1
26                                                  4
3          PASSENGER CAR                           60
34         INCOMPLETE VEHICLE                       1
4          MULTIPURPOSE PASSENGER VEHICLE (MPV)     1
           PASSENGER CAR                            6
40                                                  2
           INCOMPLETE VEHICLE                       2
           TRAILER                                  1
           TRUCK                                   33
41         INCOMPLETE VEHICLE                      58
           PASSENGER CAR                            2
           TRUCK                                    2
42                                                  1
           INCOMPLETE VEHICLE                      58
           TRUCK                                    3
43         INCOMPLETE VEHICLE                       6
           TRUCK                                    3
44         INCOMPLETE VEHICLE                       2
           MULTIPURPOSE PASSENGER VEHICLE (MPV)     1
           TRUCK                                  119
45         INCOMPLETE VEHICLE                       1
           TRUCK                                    1
46                                                  1
           INCOMPLETE VEHICLE                       3
           TRUCK                                    1
47         INCOMPLETE VEHICLE                      16
           TRUCK                                    1
48         INCOMPLETE VEHICLE                      12
           TRUCK                                    3
49                                                  1
           BUS                                     11
           INCOMPLETE VEHICLE                      64
           MULTIPURPOSE PASSENGER VEHICLE (MPV)    29
           TRUCK                                  190
5          BUS                                     74
           INCOMPLETE VEHICLE                      86
           MULTIPURPOSE PASSENGER VEHICLE (MPV)  3523
           PASSENGER CAR                          134
           TRUCK                                   96
50         BUS                                      1
           INCOMPLETE VEHICLE                      30
           MULTIPURPOSE PASSENGER VEHICLE (MPV)     2
           PASSENGER CAR                            1
           TRUCK                                    2
57         TRAILER                                  2
6                                                   4
           INCOMPLETE VEHICLE                       1
           MULTIPURPOSE PASSENGER VEHICLE (MPV)   111
           PASSENGER CAR                         7033
60         BUS                                    127
           INCOMPLETE VEHICLE                      50
           TRAILER                                  5
           TRUCK                                    1
61         PASSENGER CAR                            2
63         MULTIPURPOSE PASSENGER VEHICLE (MPV)   183
           PASSENGER CAR                          959
           TRUCK                                    5
64         TRUCK                                    1
65         INCOMPLETE VEHICLE                       2
66         MOTORCYCLE                               1
7                                                   4
           INCOMPLETE VEHICLE                       1
           MULTIPURPOSE PASSENGER VEHICLE (MPV)     3
           PASSENGER CAR                          476
           TRUCK                                    1
8          INCOMPLETE VEHICLE                       1

In [ ]:
complete_car_info.to_csv(r'''C:\Users\....csv''')

In [ ]:
# Compares VehicleType and BodyClass 
body_comparison = df[['VehicleType','BodyClass']]
body_comparison